# Minneapolis Food Inspections - ETL

In [1]:
# Dependencies

import pandas as pd
# import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from config import (user, password, host, port, database)

## Extraction Phase

In [2]:
# Read CSV 

food_insp_df = pd.read_csv("Food_Inspections.csv")

## Transformation Phase

In [3]:
# Drop unneeded columns

columns_to_drop = ['X', 'Y', 'HealthFacilityIDNumber',  'AddrNum', 'AddrNumSuffix',
       'StreetName', 'StreetType', 'PostDir', 'Unit', 'City', 'State',
       'County', 'ZipCode', 'APN', 'Ward', 'X_WebMercator', 'Y_WebMercator']

food_insp_df_clean = food_insp_df.drop(columns = columns_to_drop)

In [4]:
# Drop any existing NA's

food_insp_df_clean.dropna(how = 'any', inplace = True)

In [5]:
# Drop non-restaurant facilities

restaurants_df = food_insp_df_clean.loc[food_insp_df_clean["FacilityCategory"] == "RESTAURANT"]
restaurants_df.head()

,OBJECTID,FacilityCategory,BusinessName,RiskLevel,FullAddress,InspectionType,InspectionResult,DateOfInspection,InspectionIDNumber,FoodCodeText,...,ViolationPriority,FoodCodeItem,YearOfInspection,ViolationStatus,ViolationPoints,InspectionScore,ViolationIDNumber,Neighborhood,Latitude,Longitude
1,2,RESTAURANT,SOTAROL UPTOWN,1,2935 GIRARD AVE S,Routine,Complete,2018/07/18 13:50:00+00,204578,When using time only as a public health contro...,...,Priority1,MN Rule 4626.0408B,2018,Observed,4,92,625817,Lowry Hill East,44.94885,-93.29663
2,3,RESTAURANT,SOTAROL UPTOWN,1,2935 GIRARD AVE S,Routine,Complete,2018/07/18 13:50:00+00,204578,Employ a certified food protection manager (CF...,...,Priority3,MN Rule 4626.0033A,2018,Observed,0,92,681981,Lowry Hill East,44.94885,-93.29663
3,4,RESTAURANT,SOTAROL UPTOWN,1,2935 GIRARD AVE S,Routine,Complete,2018/07/18 13:50:00+00,204578,Food packages must be in good condition and mu...,...,Priority2,MN Rule 4626.0190,2018,Observed,2,92,681984,Lowry Hill East,44.94885,-93.29663
4,5,RESTAURANT,SOTAROL UPTOWN,1,2935 GIRARD AVE S,Routine,Complete,2018/07/18 13:50:00+00,204578,Thaw TCS food by one of the following methods:...,...,Priority3,MN Rule 4626.0380ABC,2018,Observed,0,92,682062,Lowry Hill East,44.94885,-93.29663
5,6,RESTAURANT,SOTAROL UPTOWN,1,2935 GIRARD AVE S,Routine,Complete,2019/07/18 11:45:00+00,254817,Provide a sign or poster at all handwashing si...,...,Priority3,MN Rule 4626.1457,2019,Corrected on-site,0,100,730395,Lowry Hill East,44.94885,-93.29663


In [6]:
# Drop details of inspections and violations, won't be needed, as well as Restaurant Category

inspection_details = ['FoodCodeText', 'FacilityCategory', 'RiskLevel', 'InspectorComments',
       'InspectionResult', 'ViolationPriority', 'FoodCodeItem', 'YearOfInspection',
       'ViolationStatus', 'ViolationPoints', 'ViolationIDNumber']

restaurants_df_clean = restaurants_df.drop(columns = inspection_details)
restaurants_df_clean.head(10)

,OBJECTID,BusinessName,FullAddress,InspectionType,DateOfInspection,InspectionIDNumber,InspectionScore,Neighborhood,Latitude,Longitude
1,2,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2018/07/18 13:50:00+00,204578,92,Lowry Hill East,44.94885,-93.29663
2,3,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2018/07/18 13:50:00+00,204578,92,Lowry Hill East,44.94885,-93.29663
3,4,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2018/07/18 13:50:00+00,204578,92,Lowry Hill East,44.94885,-93.29663
4,5,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2018/07/18 13:50:00+00,204578,92,Lowry Hill East,44.94885,-93.29663
5,6,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2019/07/18 11:45:00+00,254817,100,Lowry Hill East,44.94885,-93.29663
6,7,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2019/07/18 11:45:00+00,254817,100,Lowry Hill East,44.94885,-93.29663
7,8,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2019/07/18 11:45:00+00,254817,100,Lowry Hill East,44.94885,-93.29663
8,9,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2019/07/18 11:45:00+00,254817,100,Lowry Hill East,44.94885,-93.29663
9,10,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2019/07/18 11:45:00+00,254817,100,Lowry Hill East,44.94885,-93.29663
10,11,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2019/07/18 11:45:00+00,254817,100,Lowry Hill East,44.94885,-93.29663


In [7]:
# Drop duplicate inspection ID numbers (each inspection has a unique ID)

restaurants_df_unique = restaurants_df_clean.drop_duplicates("InspectionIDNumber")
restaurants_df_unique.head()

,OBJECTID,BusinessName,FullAddress,InspectionType,DateOfInspection,InspectionIDNumber,InspectionScore,Neighborhood,Latitude,Longitude
1,2,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2018/07/18 13:50:00+00,204578,92,Lowry Hill East,44.94885,-93.29663
5,6,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2019/07/18 11:45:00+00,254817,100,Lowry Hill East,44.94885,-93.29663
15,16,CRISP & GREEN,428 WASHINGTON AVE N,Routine,2019/11/08 12:00:00+00,279484,84,North Loop,44.98619,-93.27411
25,26,LUCY BAR & RESTAURANT LIMITED LIABILITY COMPANY,3915 HIAWATHA AVE,Routine,2018/10/11 10:55:00+00,228096,82,Howe,44.93168,-93.22639
38,39,LUCY BAR & RESTAURANT LIMITED LIABILITY COMPANY,3915 HIAWATHA AVE,Routine,2019/10/11 13:35:00+00,275485,80,Howe,44.93168,-93.22639


In [8]:
# Rename and rearrange columns (OBJECTID is going to be our primary key column)
## Verify that OBJECTID has unique values

print(restaurants_df_unique.shape) # Dataframe dimensions
print(restaurants_df_unique["OBJECTID"].nunique()) # Number of unique values in the OBJECTID column

(4540, 10)
4540


In [9]:
## Since the amount of unique values is the same as the number of rows, this means we can use this column as our primary key
## Rename the "OBJECTID" column to "id" and rename rest of columns to snake case, this is to comply with PostgreSQL standards
## We will also drop the Inspection ID column because it's not needed at this point

new_column_names = {"OBJECTID": "id", 
                    "BusinessName": "business_name",
                    "FullAddress": "address",
                    "InspectionType": "inspection_type",
                    "DateOfInspection": "inspection_date",
                    "InspectionScore": "inspection_score",
                    "Neighborhood": "neighborhood",
                    "Latitude": "latitude",
                    "Longitude": "longitude"}

restaurants_df_lower = restaurants_df_unique.rename(columns = new_column_names).drop(columns = "InspectionIDNumber")
restaurants_df_lower.head()

,id,business_name,address,inspection_type,inspection_date,inspection_score,neighborhood,latitude,longitude
1,2,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2018/07/18 13:50:00+00,92,Lowry Hill East,44.94885,-93.29663
5,6,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2019/07/18 11:45:00+00,100,Lowry Hill East,44.94885,-93.29663
15,16,CRISP & GREEN,428 WASHINGTON AVE N,Routine,2019/11/08 12:00:00+00,84,North Loop,44.98619,-93.27411
25,26,LUCY BAR & RESTAURANT LIMITED LIABILITY COMPANY,3915 HIAWATHA AVE,Routine,2018/10/11 10:55:00+00,82,Howe,44.93168,-93.22639
38,39,LUCY BAR & RESTAURANT LIMITED LIABILITY COMPANY,3915 HIAWATHA AVE,Routine,2019/10/11 13:35:00+00,80,Howe,44.93168,-93.22639


In [10]:
# Split date from time in inspection_date column, we only need the date component

inspection_dates = restaurants_df_lower["inspection_date"]
new_insp_dates = [date.split(" ")[0] for date in inspection_dates]
restaurants_df_with_dates = restaurants_df_lower.assign(inspection_date = new_insp_dates)
restaurants_df_with_dates

,id,business_name,address,inspection_type,inspection_date,inspection_score,neighborhood,latitude,longitude
1,2,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2018/07/18,92,Lowry Hill East,44.94885,-93.29663
5,6,SOTAROL UPTOWN,2935 GIRARD AVE S,Routine,2019/07/18,100,Lowry Hill East,44.94885,-93.29663
15,16,CRISP & GREEN,428 WASHINGTON AVE N,Routine,2019/11/08,84,North Loop,44.98619,-93.27411
25,26,LUCY BAR & RESTAURANT LIMITED LIABILITY COMPANY,3915 HIAWATHA AVE,Routine,2018/10/11,82,Howe,44.93168,-93.22639
38,39,LUCY BAR & RESTAURANT LIMITED LIABILITY COMPANY,3915 HIAWATHA AVE,Routine,2019/10/11,80,Howe,44.93168,-93.22639
...,...,...,...,...,...,...,...,...,...
42557,42558,TONY'S DINER,1327 4TH ST SE,Routine,2021/04/02,92,Marcy Holmes,44.98097,-93.23602
42574,42575,PAPA JOHNS PIZZA,4618 NICOLLET AVE,Routine,2021/04/14,98,Tangletown,44.91883,-93.27836
42596,42597,STELLA'S FISH CAFE & PRESTIGE OYSTER BAR,1400 LAKE ST W,Routine,2019/07/25,100,Lowry Hill East,44.94867,-93.29726
42605,42606,CRAVE CAFE,2100 SUMMER ST NE,Routine,2021/04/14,100,Mid - City Industrial,44.99603,-93.23046


## Loading Phase

In [11]:
# Create Postgres engine
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

In [12]:
# Confirm connection was established

## Declare base using `automap_base()`
Base = automap_base()

## Use the Base class to reflect the database tables
Base.prepare(engine, reflect = True)

## Print all of the classes mapped to the Base
Base.classes.keys()

['food_inspections']

In [13]:
# Load to SQL using Pandas' to_sql function
restaurants_df_with_dates.to_sql(name = "food_inspections", con = engine, if_exists = 'append', index = False)